# Лабораторная работа 3. TF-IDF взвешивание терминов для векторизации текстов

**Задание 1.** Реализуйте векторизацию текстов на основе tf-idf метода взвешивания терминов. Предусмотрите задание размера словаря терминов, используемого для взвешивания. Можно код организовать отдельным модулем. Протестируйте свою реализацию метода.

**Задание 2.** Сделайте классификацию новостных текстов из предыдущей лабораторной работы. Векторизация tf-idf имеет два этапа: построение словаря и построение векторов, поэтому исходный датасет надо разделить на две части. Первая часть будет использоваться для формирования словаря, а вторая часть для обучения и проверки модели классификации.

Основные действия:
- предобработка (токенизация, удаление пунктуации, лематизация),
- разбиение на выборку текстов на две части в отношении 1:1 (первую используйте для построения словаря, а вторую часть - для построения модели классификации),
- построение словаря для tf-idf векторизации по первой части выборки (**на основе вашей реализации tf-idf**),
- векторизация текстов второй части выборки (**на основе вашей реализации tf-idf**),
- классификация текстов по темам на основе **логистической регрессии** (вторую часть выборки надо снова разделит две части:обучающую и валидационную),
- оценивание качество классификации.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from giezz.lab3 import tfidf
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc
)
import re

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

def preprocess_text_natasha(text):
    if not isinstance(text, str):
        return ""

    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    tokens = []
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        lemma = token.lemma.lower().strip()
        if len(lemma) > 2 and not lemma.isdigit():
            tokens.append(lemma)

    return " ".join(tokens)

df = pd.read_csv('../../labs/lenta_ru_news_filtered.csv')
df['processed_text'] = df['text'].apply(preprocess_text_natasha)
df = df[df['processed_text'].str.len() > 0]
df_for_dict, df_for_classifier = train_test_split(df, test_size=0.5, random_state=42, stratify=df['topic'])

print(f"Размер первой части (для словаря): {len(df_for_dict)}")
print(f"Размер второй части (для классификации): {len(df_for_classifier)}")

vectorizer = tfidf.TfidfVectorizer(max_features=500)
vectorizer.fit(df_for_dict['processed_text'].tolist())

print(f"Размер словаря: {len(vectorizer.vocabulary)}")

X_second = vectorizer.transform(df_for_classifier['processed_text'].tolist())
y_second = df_for_classifier['topic'].values

X_train, X_val, y_train, y_val = train_test_split(X_second, y_second, test_size=0.3, random_state=42, stratify=y_second)

print(f"Обучающая выборка: {X_train.shape[0]} примеров")
print(f"Валидационная выборка: {X_val.shape[0]} примеров")

model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_val, y_pred))

plt.figure(figsize=(12, 6))
topic_counts = pd.Series(y_val).value_counts()
topic_accuracy = {}

for topic in topic_counts.index:
    mask = y_val == topic
    if mask.sum() > 0:
        topic_accuracy[topic] = accuracy_score(y_val[mask], y_pred[mask])

plt.subplot(1, 2, 1)
topic_counts.plot(kind='bar', title='Распределение тем в валидационной выборке')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
pd.Series(topic_accuracy).plot(kind='bar', title='Accuracy по темам')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

**Задание 3.** Используя модуль TfIdfVectorizer библиотеки sklearn, сделайте классификацию новостных текстов из предыдущей лабораторной работы. Предусмотрите предобработку текстов и задание ограничение словаря при взвешивании. Оцените качество классификации.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

sklearn_vectorizer = TfidfVectorizer(max_features=500)
X_second_sklearn = sklearn_vectorizer.fit_transform(df_for_classifier['processed_text'])


X_train_sk, X_val_sk, y_train_sk, y_val_sk = train_test_split(
    X_second_sklearn, y_second, test_size=0.3, random_state=42, stratify=y_second
)

model_sk = LogisticRegression(random_state=42, max_iter=1000)
model_sk.fit(X_train_sk, y_train_sk)

y_pred_sk = model_sk.predict(X_val_sk)

accuracy_sk = accuracy_score(y_val_sk, y_pred_sk)
print(f"Accuracy (sklearn): {accuracy_sk:.4f}")
print("\nClassification Report (sklearn):")
print(classification_report(y_val_sk, y_pred_sk))

**Задание 4.** Постройте график зависимости значения `accuracy` от размера словаря, по которому выполняется tf-idf векторизация. Для этого используйте первую и вторую выборки исходного датасета, которые были получены при первом разбиении в предыдущем задании. По первой части текстов один раз строите словарь `D_common`, потом из него формируете словарь `D_k` размером `k`. По словарю `D_k` векторизуете вторую часть текстов, затем по которой обучаете и проверяете качество модели. Значение `k` меняется от [500, 1000, 1500, 2000, 3000, 5000].

In [ ]:
k_values = [500, 1000, 1500, 2000, 3000, 5000]
accuracies = []

for k in k_values:
    print(f"\nРазмер словаря: {k}")
    vectorizer_k = tfidf.TfidfVectorizer(max_features=k)
    vectorizer_k.fit(df_for_dict['processed_text'].tolist())
    X_k = vectorizer_k.transform(df_for_classifier['processed_text'].tolist())
    X_train_k, X_val_k, y_train_k, y_val_k = train_test_split(
        X_k, y_second, test_size=0.3, random_state=42, stratify=y_second
    )

    model_k = LogisticRegression(random_state=42, max_iter=1000)
    model_k.fit(X_train_k, y_train_k)

    y_pred_k = model_k.predict(X_val_k)
    accuracy_k = accuracy_score(y_val_k, y_pred_k)
    accuracies.append(accuracy_k)

    print(f"Accuracy для k={k}: {accuracy_k:.4f}")


plt.figure(figsize=(10, 6))
plt.plot(k_values, accuracies, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Размер словаря (k)')
plt.ylabel('Accuracy')
plt.title('Зависимость accuracy от размера словаря TF-IDF')
plt.grid(True, alpha=0.3)

for i, (k, acc) in enumerate(zip(k_values, accuracies)):
    plt.annotate(f'{acc:.3f}', (k, acc), textcoords="offset points",
                 xytext=(0,10), ha='center')

plt.tight_layout()
plt.show()

results_df = pd.DataFrame({
    'Размер словаря': k_values,
    'Accuracy': accuracies
})
print("\nРезультаты:")
print(results_df.to_string(index=False))
best_idx = np.argmax(accuracies)
print(f"\nЛучший результат: k={k_values[best_idx]}, accuracy={accuracies[best_idx]:.4f}")

**Задание 5.** Реализуйте поиск новостных текстов по текстовому запросу, задаваемому пользователем, на основе tf-idf векторизации (можно использовать любую реализацию методов).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

full_vectorizer = TfidfVectorizer(max_features=5000)
full_tfidf_matrix = full_vectorizer.fit_transform(df['processed_text'])

def search_news(query, vectorizer, tfidf_matrix, original_df, top_k=5):
    processed_query = preprocess_text_natasha(query)
    query_vector = vectorizer.transform([processed_query])
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_indices = similarities.argsort()[-top_k:][::-1]
    results = []
    for i, idx in enumerate(top_indices):
        if similarities[idx] > 0:
            results.append({
                'rank': i + 1,
                'title': original_df.iloc[idx]['title'],
                'topic': original_df.iloc[idx]['topic'],
                'similarity': round(similarities[idx], 4),
                'text_preview': original_df.iloc[idx]['text'][:200] + '...' if len(original_df.iloc[idx]['text']) > 200 else original_df.iloc[idx]['text']
            })

    return results

In [ ]:
user_query = "поставки спирта".strip()
print(f"\nРезультаты поиска по запросу: '{user_query}'")
search_results = search_news(user_query, full_vectorizer, full_tfidf_matrix, df, top_k=5)

if search_results:
    for result in search_results:
        print(f"\n{result['rank']}. [{result['topic']}] {result['title']}")
        print(f"   Схожесть: {result['similarity']}")
        print(f"   Фрагмент: {result['text_preview']}")
else:
    print("по вашему запросу ничего не найдено.")